In [2]:
import pickle
import sys
from collections import defaultdict
import dill

Załadowanie plików zawierających częstości występowania dla modelu jedno- oraz dwugramowego:

In [3]:
with open('../pickles/one_gram_model.p', 'rb') as file:
    one_gram_dict = pickle.load(file)

In [4]:
with open('../pickles/two_gram_model.p', 'rb') as file:
    two_gram_dict = pickle.load(file)

Przekształcam słownik dwugramowy w formę podwójnych kluczy:

{'się w' : 100,...} -> {'się' {'w': 100}}

In [10]:
two_gram_dict_two_step = defaultdict(int)
isinstance(two_gram_dict_two_step['się'], dict)

False

In [5]:
two_gram_dict_two_step = defaultdict(int)

for bigram, count in two_gram_dict.items():
    first, second = bigram.split(' ')
    try:
        two_gram_dict_two_step[first][second] += count
    except TypeError:
        two_gram_dict_two_step[first] = defaultdict(int)
        two_gram_dict_two_step[first][second] += count


Poniższy przykład pokazuje, że oba modele (1- oraz 2-gramowy) należy oszacować oddzielnie. Ynnymy słowy prawdopodbieństwo wystąpienia jednego ze słów z modelu dwugramowego nie może zostać oszacowane na podstawie jego wartości w modelu jednogramowym:

In [12]:
one_gram_dict['się']

4332853

In [6]:
sum(two_gram_dict_two_step['się'].values())

4079294

In [7]:
two_gram_dict_two_step_probs = defaultdict(float)

for first_key, dictionary in two_gram_dict_two_step.items():
    two_gram_dict_two_step_probs[first_key] = defaultdict(float)
    for second_key, count in two_gram_dict_two_step[first_key].items():
        denominator = sum(two_gram_dict_two_step[first_key].values())
        prob = count / denominator
        two_gram_dict_two_step_probs[first_key][second_key] = prob
    

Zapisanie modelu do pliku:

In [20]:
with open('../pickles/two_gram_probs.p', 'wb') as file:
    pickle.dump(two_gram_dict_two_step_probs, file, protocol=pickle.HIGHEST_PROTOCOL)

Używam wygładzania Laplace'a. Słownik inicjuję wartością słowa, które wystąpiło tylko raz. Jako, że jest wygładzanie to słowo, które wystąpiło tylko raz to tak naprawdę słowo, które nie wystąpiło wcale:

In [72]:
no_of_words = len(one_gram_dict.keys())
basic_denominator = sum(one_gram_dict.values()) #denominator before smoothing being applied

default_value = float(1/(no_of_words+basic_denominator))

one_gram_dict_probs = defaultdict(lambda: default_value)

for word, count in one_gram_dict.items():
    one_gram_dict_probs[word] = (count+1) / (basic_denominator+no_of_words)

Zapisanie modelu do pliku:

In [75]:
with open('../pickles/one_gram_probs.p', 'wb') as file:
    dill.dump(one_gram_dict_probs, file, protocol=dill.HIGHEST_PROTOCOL)